In [14]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [15]:
# Load the csv exported in Part I to a DataFrame
data_path = "../WeatherPy/Output/weather.csv"

weather_data = pd.read_csv(data_path)
weather_df=pd.DataFrame(weather_data)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,paamiut,61.9940,-49.6678,-1.46,85,20,2.71,GL,1612724530
1,punta arenas,8.9897,125.3400,23.80,92,100,0.60,PH,1612724530
2,nanortalik,60.1432,-45.2371,-0.45,89,74,6.29,GL,1612724224
3,mahebourg,-20.4081,57.7000,27.00,78,75,4.63,MU,1612724531
4,busselton,-33.6500,115.3333,17.22,99,98,0.45,AU,1612724532


In [16]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations=weather_df[['Lat','Lng']]
humidity=weather_data['Humidity'].astype(float)

In [17]:
weather_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,561.000000,561.000000,561.000000,561.000000,561.000000,561.000000,5.610000e+02
mean,21.665671,23.290651,10.152299,70.918004,54.907308,3.926435,1.612725e+09
std,31.839389,86.816830,17.368789,21.310302,41.338956,2.818304,2.225793e+02
min,-54.800000,-179.166700,-42.000000,3.000000,0.000000,0.140000,1.612724e+09
25%,-3.736100,-49.667800,-1.140000,60.000000,3.000000,1.950000,1.612725e+09
50%,25.533300,25.136100,15.560000,77.000000,75.000000,3.160000,1.612725e+09
75%,47.804300,95.683300,24.890000,86.000000,99.000000,5.140000,1.612725e+09
max,78.218600,178.416700,35.400000,100.000000,100.000000,21.610000,1.612725e+09


In [21]:
# Plot heatmap
fig=gmaps.figure()

# Add Heatmap layer to map and show figure
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,
                               dissipating=False,max_intensity=100,
                               point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
# Narrow down cities to fit weather conditions
conditions_df=weather_df
conditions_df = conditions_df.loc[(conditions_df["Max Temp"] < 26) & (conditions_df["Max Temp"] > 22)]
conditions_df=conditions_df.loc[conditions_df['Wind Speed']<6]
conditions_df=conditions_df.loc[conditions_df['Cloudiness']==0]
conditions_df=conditions_df.loc[conditions_df['Humidity']<70]


# Drop all rows with null values
conditions_df=conditions_df.dropna()

# Reset index and display DataFrame
conditions_df.head()
# len(conditions_df)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,castro,-24.7911,-50.0119,24.58,47,0,1.55,BR,1612724154
34,biltine,14.5333,20.9167,23.12,17,0,5.17,TD,1612724544
55,santiago de cao,-33.4569,-70.6483,25.56,60,0,2.57,CL,1612724404
347,cidreira,-30.1811,-50.2056,24.37,62,0,4.59,BR,1612724694
450,mecca,21.4267,39.8261,22.82,31,0,2.71,SA,1612724739


In [23]:
# Store into variable named hotel_df
hotel_df=conditions_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
31,castro,-24.7911,-50.0119,24.58,47,0,1.55,BR,1612724154
34,biltine,14.5333,20.9167,23.12,17,0,5.17,TD,1612724544
55,santiago de cao,-33.4569,-70.6483,25.56,60,0,2.57,CL,1612724404
347,cidreira,-30.1811,-50.2056,24.37,62,0,4.59,BR,1612724694
450,mecca,21.4267,39.8261,22.82,31,0,2.71,SA,1612724739
528,phitsanulok,16.9167,100.5000,22.97,46,0,1.90,TH,1612724771


In [24]:
# # Add a "Hotel Name" column to the DataFrame
# header_list = ['Hotel Name','City','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed','Country','Date']
# hotel_df= hotel_df.reindex(columns = header_list) 
# hotel_df.head()

In [25]:
# Set parameters to search for hotels within 5000 meter radius
hotels= []

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 31: castro.
Closest hotel is Castro.
------------
Retrieving Results for Index 34: biltine.
Closest hotel is Biltine.
------------
Retrieving Results for Index 55: santiago de cao.
Closest hotel is Santiago.
------------
Retrieving Results for Index 347: cidreira.
Closest hotel is CIDREIRA.
------------
Retrieving Results for Index 450: mecca.
Closest hotel is Mecca.
------------
Retrieving Results for Index 528: phitsanulok.
Closest hotel is Wang Nok Aen.
------------


In [26]:
# Store into variable named hotel_df
hotel_df=conditions_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
31,castro,-24.7911,-50.0119,24.58,47,0,1.55,BR,1612724154,Castro
34,biltine,14.5333,20.9167,23.12,17,0,5.17,TD,1612724544,Biltine
55,santiago de cao,-33.4569,-70.6483,25.56,60,0,2.57,CL,1612724404,Santiago
347,cidreira,-30.1811,-50.2056,24.37,62,0,4.59,BR,1612724694,CIDREIRA
450,mecca,21.4267,39.8261,22.82,31,0,2.71,SA,1612724739,Mecca
528,phitsanulok,16.9167,100.5000,22.97,46,0,1.90,TH,1612724771,Wang Nok Aen


In [29]:
# # Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]
markers = gmaps.marker_layer(locations)


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# Create hotel symbol layer on top of heatmap
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)


# Add heat layer
heat_layer=gmaps.heatmap_layer(locations,
                               dissipating=False,max_intensity=100,
                               point_radius=4)

# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))